## Testing that polymerist is importable

In [1]:
import logging
logging.basicConfig(
    level=logging.INFO
)

import polymerist as ps
from polymerist.genutils.importutils import module_hierarchy

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:rdkit:Enabling RDKit 2023.09.5 jupyter extensions


[16:45:09] WARNING: not removing hydrogen atom with dummy atom neighbors
INFO:polymerist.smileslib.functgroups:Loading functional group SMARTS data from LUT


In [2]:
import MDAnalysis as mda

import warnings
# suppress some MDAnalysis warnings when writing PDB files
warnings.filterwarnings('ignore')

In [ ]:
from pathlib import Path
from polymerist.genutils.fileutils.pathutils import assemble_path
import os

## Parameterizing PDB system

### Initializing paths and working directory

In [4]:
cwd=os.getcwd()
working_dir =Path(cwd+'/HP_35_original').mkdir(parents=True, exist_ok=True)
mol_name = 'HP_35m'

pdb_path = assemble_path(working_dir, mol_name, extension='pdb') 
sdf_path = assemble_path(working_dir, mol_name, extension='sdf') 
inc_path = assemble_path(working_dir, mol_name, extension='pkl') 

TypeError: unsupported operand type(s) for /: 'NoneType' and 'str'

### Load Topology from PDB ( no overhead since not using long molecules), check that molecules exist

In [5]:
from openff.toolkit import Molecule, Topology, ForceField
from polymerist.mdtools.openfftools import topology


if sdf_path.exists():
    offtop = topology.topology_from_sdf(sdf_path)
else:
    assert(pdb_path.exists())
    offtop = Topology.from_pdb(pdb_path)

### Parameterize system w/ OpenFF

In [6]:
import pickle
from openff.interchange import Interchange

from polymerist.unitutils.interop import openff_to_openmm
from polymerist.mdtools.openfftools import boxvectors
from polymerist.mdtools.openfftools import TKREGS, FF_PATH_REGISTRY


# load interchange from file if already extant, otherwise make a new one and save it
if inc_path.exists():
    with inc_path.open('rb') as file:
        inc  = pickle.load(file)
else:
    ff = ForceField('openff-2.1.0.offxml', 'tip3p.offxml') # load generic Sage + TIP3P force fields
    inc = ff.create_interchange(topology=offtop, toolkit_registry=TKREGS['OpenEye Toolkit']) # convert to interchange for export
    inc.box = boxvectors.get_topology_bbox(offtop)

    if not sdf_path.exists():
        topology.topology_to_sdf(sdf_path, inc.topology)

    with inc_path.open('wb') as file:
        pickle.dump(inc, file)

# extract OpenMM-specific objects to initialize simulations later
ommtop = inc.to_openmm_topology()
ommsys = inc.to_openmm_system(combine_nonbonded_forces=False, add_constrained_forces=True)
ommpos = openff_to_openmm(inc.positions)

INFO:openff.toolkit.typing.engines.smirnoff.parameters:Attempting to up-convert vdW section from 0.3 to 0.4
INFO:openff.toolkit.typing.engines.smirnoff.parameters:Successfully up-converted vdW section from 0.3 to 0.4. `method="cutoff"` is now split into `periodic_method="cutoff"` and `nonperiodic_method="no-cutoff"`.
INFO:openff.toolkit.typing.engines.smirnoff.parameters:Attempting to up-convert Electrostatics section from 0.3 to 0.4
INFO:openff.toolkit.typing.engines.smirnoff.parameters:Successfully up-converted Electrostatics section from 0.3 to 0.4. `method="PME"` is now split into `periodic_potential="Ewald3D-ConductingBoundary"`, `nonperiodic_potential="Coulomb"`, and `exception_potential="Coulomb"`.
INFO:polymerist.mdtools.openfftools.topology:Successfully serialized SDF Topology to /home/bamo6610/Documents/openff-dev/HW_openMM/HW_3m_JC.sdf


### Create ion subsystem

## Applying constraints - Harmonic Potential

In [7]:
from openmm.unit import kilojoule_per_mole, nanometer
from openmm import CustomExternalForce

k = 0.68095403*(kilojoule_per_mole/nanometer**2) # define force constant
z_center = 7.2 * nanometer

fb_force = CustomExternalForce('0.5*k*((z-z0)^2)') # apply harmonic potential restraints
fb_force.addGlobalParameter('k', k)
fb_force.addGlobalParameter('z0', z_center)
ommsys.addForce(fb_force)

7

In [8]:
from openmm.app.topology import Topology

for atom in ommtop.atoms(): # apply costum external force only to ions in system
    res=atom.residue.name
    if (res == 'NA') or (res == 'CL'):
        fb_force.addParticle(
            atom.index,
            []# if (res == 'NA') or (res == 'CL') else []
        )

## Creating OpenMM simulations

### Defining simulation parameters

In [9]:
from polymerist.mdtools.openmmtools.parameters import SimulationParameters, ThermoParameters, IntegratorParameters, ReporterParameters
from polymerist.mdtools.openmmtools.thermo import EnsembleFactory

from openmm.unit import kelvin, atmosphere, nanosecond, picosecond, femtoseconds


all_omm_sims : dict[str, SimulationParameters] = {
    'equil_sim' : SimulationParameters(
        integ_params=IntegratorParameters(
            time_step=2*femtoseconds,
            total_time=2*nanosecond,
            num_samples=50,
        ),
        thermo_params=ThermoParameters(
            ensemble='NVT',
            temperature=300 * kelvin,
        ),
        reporter_params=ReporterParameters(
            traj_ext='dcd',
        ),
    ),
    'prod_sim' : SimulationParameters(
        integ_params=IntegratorParameters(
            time_step=2*femtoseconds,
            total_time=20*nanosecond,
            num_samples=250,
        ),
        thermo_params=ThermoParameters(
            ensemble='NVT',
            temperature=300 * kelvin,
        ),
        reporter_params=ReporterParameters(),
    ),
}    

### Run the defined simulation in series

In [10]:
from polymerist.mdtools.openmmtools.execution import run_simulation_schedule


omm_sim_dir = Path('HP_35_original/OpenMM_HP_35_original')
omm_sim_dir.mkdir(exist_ok=True)

history = run_simulation_schedule(
    working_dir=omm_sim_dir,
    schedule=all_omm_sims,
    init_top=ommtop,
    init_sys=ommsys,
    init_pos=ommpos,
    return_history=True
)

INFO:polymerist.mdtools.openmmtools.execution:Initializing simulation 1/2 ("equil_sim")
INFO:polymerist.mdtools.openmmtools.thermo:Created LangevinMiddleIntegrator for NVT (Canonical) ensemble
INFO:polymerist.mdtools.openmmtools.reporters:Prepared DCDReporter which reports to OpenMM_sims_HPs/equil_sim/equil_sim_trajectory.dcd
INFO:polymerist.mdtools.openmmtools.reporters:Prepared CheckpointReporter which reports to OpenMM_sims_HPs/equil_sim/equil_sim_checkpoint.chk
INFO:polymerist.mdtools.openmmtools.reporters:Prepared StateReporter which reports to OpenMM_sims_HPs/equil_sim/equil_sim_state.xml
INFO:polymerist.mdtools.openmmtools.reporters:Prepared StateDataReporter which reports to OpenMM_sims_HPs/equil_sim/equil_sim_state_data.csv
INFO:polymerist.mdtools.openmmtools.preparation:Setting positions in Context
INFO:polymerist.mdtools.openmmtools.execution:Performing energy minimization (initial PE = 25821345213.2678 kJ/mol)
INFO:polymerist.mdtools.openmmtools.execution:Energy successfull

In [11]:
from polymerist.mdtools.openmmtools.description import describe_forces

sim_post = history['equil_sim']['simulation']
desc_str = describe_forces(sim_post.system, as_str=True)
print(desc_str)

'vdW force'
	'CutoffDistance' : Quantity(value=0.9, unit=nanometer)
	'EnergyFunction' : '4*epsilon*((sigma/r)^12-(sigma/r)^6); sigma=(sigma1+sigma2)/2; epsilon=sqrt(epsilon1*epsilon2); '
	'ForceGroup' : 0
	'Name' : 'vdW force'
	'NonbondedMethod' : 2
	'NumComputedValues' : 0
	'NumEnergyParameterDerivatives' : 0
	'NumExclusions' : 33300
	'NumFunctions' : 0
	'NumGlobalParameters' : 0
	'NumInteractionGroups' : 0
	'NumParticles' : 33676
	'NumPerParticleParameters' : 2
	'NumTabulatedFunctions' : 0
	'SwitchingDistance' : Quantity(value=0.8, unit=nanometer)
	'UseLongRangeCorrection' : True
	'UseSwitchingFunction' : True
	'Type' : 'CustomNonbondedForce'
	2 : 'CutoffPeriodic'
'Electrostatics force'
	'CutoffDistance' : Quantity(value=0.9, unit=nanometer)
	'EwaldErrorTolerance' : 0.0001
	'ExceptionsUsePeriodicBoundaryConditions' : False
	'ForceGroup' : 1
	'IncludeDirectSpace' : True
	'LJPMEParameters' : [0.0, 0, 0, 0]
	'Name' : 'Electrostatics force'
	'NonbondedMethod' : 4
	'NumExceptionParameterO